<a href="https://colab.research.google.com/github/maryamelnahas/CSEN711-MS2-Cryptography-and-Encryption/blob/main/CSEN711_MS2_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import math
import os
from google.colab import files
import hashlib
import secrets
import getpass
import re

**Loading Dataset**

In [4]:
dataset = pd.read_csv('production.tsv', sep='\t', header=None, names=['raw_password'])
dataset.head()

/tmp/ipython-input-1448325701.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('production.tsv', sep='\t', header=None, names=['raw_password'])


,raw_password
password,strength
5Ka392f0c29,1
97DMs07gBQ3qP9Hwf3YxJxY3yFvoh1W9b6HrmSNfOB9rjUIZJJX7g0mrv0Rlv51r184odr317jq0df3MBIJ4B07kMK7JCx4QG7,2
qD2bN8VmEB,1
rJu42Kx,1


**Playfair Cipher**

In [5]:
def prepare_text(text):
    text = str(text).upper().replace("J", "I")
    return re.sub(r'[^A-Z]', '', text)

def generate_key_matrix(key):
    key = prepare_text(key)
    alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ" # 'J' is omitted
    matrix = []
    used_chars = set()

    for char in key:
        if char not in used_chars and char in alphabet:
            matrix.append(char)
            used_chars.add(char)

    for char in alphabet:
        if char not in used_chars:
            matrix.append(char)
            used_chars.add(char)

    return [matrix[i:i+5] for i in range(0, 25, 5)]

def find_position(matrix, char):
    for r, row in enumerate(matrix):
        if char in row:
            return r, row.index(char)
    return None

def preprocess_digraphs(text):
    text = prepare_text(text)
    digraphs = []
    i = 0
    while i < len(text):
        a = text[i]
        b = text[i+1] if i+1 < len(text) else 'X'

        if a == b:
            digraphs.append((a, 'X'))
            i += 1
        else:
            digraphs.append((a, b))
            i += 2
    return digraphs

def encrypt_playfair_logic(plaintext, key):
    matrix = generate_key_matrix(key)
    digraphs = preprocess_digraphs(plaintext)
    ciphertext = []

    for a, b in digraphs:
        r1, c1 = find_position(matrix, a)
        r2, c2 = find_position(matrix, b)

        if r1 == r2: # Same Row: Shift Right
            ciphertext.append(matrix[r1][(c1 + 1) % 5])
            ciphertext.append(matrix[r2][(c2 + 1) % 5])
        elif c1 == c2: # Same Column: Shift Down
            ciphertext.append(matrix[(r1 + 1) % 5][c1])
            ciphertext.append(matrix[(r2 + 1) % 5][c2])
        else: # Rectangle: Swap Columns
            ciphertext.append(matrix[r1][c2])
            ciphertext.append(matrix[r2][c1])

    return "".join(ciphertext)


PLAYFAIR_KEY = "BUSINESSINFORMATICS"
dataset['playfair_encrypted'] = dataset['raw_password'].apply(
    lambda x: encrypt_playfair_logic(str(x), PLAYFAIR_KEY)
)

print(f"Playfair Encryption Complete using key: {PLAYFAIR_KEY}")
dataset.head()

Playfair Encryption Complete using key: BUSINESSINFORMATICS


,raw_password,playfair_encrypted
password,strength,UCMFMQAK
5Ka392f0c29,1,
97DMs07gBQ3qP9Hwf3YxJxY3yFvoh1W9b6HrmSNfOB9rjUIZJJX7g0mrv0Rlv51r184odr317jq0df3MBIJ4B07kMK7JCx4QG7,2,
qD2bN8VmEB,1,
rJu42Kx,1,
